![](images/roboticsdevday.jpeg)


## CREATING CUSTOM GAZEBO PLUGIN

---
#### Author Details:

* ***Name: Shantanu Parab***
* ***Email ID: shantanuparab99@gmail.com***
* ***[LinkedIn](https://www.linkedin.com/in/shantanu-parab/)***
* ***[Github](https://github.com/shantanuparabumd)***

---
#### Introduction

The goal of this project is to understand how to write custom Gazebo plugins and integrate them with ROS. Gazebo plugins extend the capabilities of Gazebo by allowing custom behaviors and interactions. There are different types of plugins, such as Sensor Plugins, Model Plugins, and more. In this project, we will focus on designing a Model Plugin.

We will use a simple example of a Conveyor Belt to design our plugin, which will help us grasp fundamental concepts like adding a plugin to a robot using URDF/SDF files. Additionally, we will integrate ROS functionalities such as topic subscription into our plugin.

---
#### Objectives

***Task 1: Setting Up the Package for Plugin***

How to edit the CMakeLists.txt file to build the plugin and export it to be used by other packages.

***Task 2: Establishing Connection with Gazebo***

Understand how to establish a connection between our custom plugin and the Gazebo simulation environment.

***Task 3: Accessing Model Joints***

Explore methods for accessing and manipulating the joints of a model within the Gazebo environment.

***Task 4: Accessing Model and Plugin Parameters/Arguments from URDF/SDF***

Learn how to retrieve and utilize parameters and arguments specified in URDF/SDF files for models and plugins.

***Task 5: Moving the Joints***

Implement functionality within the plugin to control and move the model's joints.

***Task 6: Adding a Topic Subscription***

Integrate ROS by adding a topic subscription to the plugin, enabling it to receive and respond to messages from ROS topics.

![](images/conveyor.gif)

------------------
### Addtional Resources

1. [Gazebo API Documentation](https://osrf-distributions.s3.amazonaws.com/gazebo/api/dev/modules.html)
2. [PIMPL Idiom C++](https://en.cppreference.com/w/cpp/language/pimpl)
3. [Drone Plugin Gazebo](https://github.com/shantanuparabumd/drone_delivery)
4. [Gazebo Plugin Basics](https://classic.gazebosim.org/tutorials?tut=ros_gzplugins)
5. [How to create Gazebo plugins in ROS](https://www.youtube.com/watch?v=LRjT_1huVKY&list=PLK0b4e05Lnzah3QAIsdb0JxAY21YypdZl)

----------------------------
### How to Launch our Simulation



##### Source ROS (Galactic) Environment

In [ ]:
source /opt/ros/galactic/setup.bash

##### Build your package

In [ ]:
colcon build --packages-select conveyor_belt

##### Source your workspace

In [ ]:
source install/setup.bash

###### Launch Gazebo Environment

In [ ]:
ros2 launch conveyor_belt robot.launch.py

###### Spawn an Object on the belt

In [ ]:
ros2 run conveyor_belt spawn_object.py

----------
### Folder Structure of our Project

We will have the following folder structure for our package. You will notice two files missing i.e 
1. **include/conveyor_belt/conveyor_belt_plugin.hpp**
2. **src/conveyor_belt_plugin.cpp**


**Expected Folder Structure**
![](images/folder_structure.png)

-----------------------------
### File to be Updated

Throught this demonstration we will be just working with one single file. Which is as follows

**File: conveyor_belt/src/conveyor_belt_plugin.cpp**

Before we move on to this file we will be making a few changes to setup as below

*************************
### Task 1: Setup Package for the Plugin 

#### Task 1.1: Create a Header File

1. In the **include/conveyor_belt** folder of your package create a file named **conveyor_belt_plugin.hpp**.
2. Add the below code to the file.

In [ ]:
#ifndef CONVEYOR_BELT_PLUGIN_HPP_
#define CONVEYOR_BELT_PLUGIN_HPP_

#include <gazebo/common/Plugin.hh>

#include <memory>

namespace gazebo
{
  
  class ConveyorBeltPluginPrivate;

  class ConveyorBeltPlugin : public gazebo::ModelPlugin
  {
  public:
    
    ConveyorBeltPlugin();

    virtual ~ConveyorBeltPlugin();

    
    void Load(gazebo::physics::ModelPtr model, sdf::ElementPtr sdf) override;

  private:
    std::unique_ptr<ConveyorBeltPluginPrivate> impl_;
  };

  #endif  // CONVEYOR_BELT_PLUGIN_HPP_
} // namespace gazebo



---
#### Task 1.2: Create a CPP (Source) File

1. In the **src** folder of your package create a file named **conveyor_belt_plugin.cpp**.
2. Add the following boiler-plate code to the file.

In [ ]:
#include <gazebo/physics/Model.hh>
#include <gazebo/physics/Joint.hh>
#include <conveyor_belt/conveyor_belt_plugin.hpp>
#include <gazebo_ros/node.hpp>
#include <rclcpp/rclcpp.hpp>
#include <std_msgs/msg/bool.hpp>



#include <memory>


#define RED "\033[1;31m"
#define GREEN "\033[1;32m"
#define BLUE "\033[1;34m"
#define PURPLE "\033[1;35m"
#define ORANGE "\033[1;33m"
#define CYAN "\033[1;36m"
#define RESET "\033[0m"


namespace gazebo{


class ConveyorBeltPluginPrivate
{
  

};


/**
 * @brief Construct a new Conveyor Belt Plugin:: Conveyor Belt Plugin object
 * 
 */
ConveyorBeltPlugin::ConveyorBeltPlugin()
: impl_(std::make_unique<ConveyorBeltPluginPrivate>())
{
  std::cout << GREEN<<"Initialized the Conveyor Belt Plugin" <<RESET << std::endl;
}

/**
 * @brief Destroy the Conveyor Belt Plugin:: Conveyor Belt Plugin object
 * 
 */
ConveyorBeltPlugin::~ConveyorBeltPlugin()
{
    std::cout << RED<<"Destroyed the Conveyor Belt Plugin" <<RESET << std::endl;
}


/**
 * @brief Load the SDF/URDF model of the robot and access the links/joints.
 * 
 * @param model 
 * @param sdf 
 */
void ConveyorBeltPlugin::Load(gazebo::physics::ModelPtr model, sdf::ElementPtr sdf)
{
    


}

// Register this plugin with the simulator
GZ_REGISTER_MODEL_PLUGIN(ConveyorBeltPlugin)

}

---
#### Task 1.3: Update the CMakeLists.txt file

1. Add the below code to the **CMkaeLists.txt** file to make the package recognize the plugin and also export it so that it can be used with other packages as well

In [ ]:
add_library(conveyor_belt_plugin SHARED
  src/conveyor_belt_plugin.cpp
)
target_include_directories(conveyor_belt_plugin PUBLIC include)
ament_target_dependencies(conveyor_belt_plugin
  "gazebo_ros"
)
ament_export_libraries(conveyor_belt_plugin)


install(TARGETS
    conveyor_belt_plugin
  ARCHIVE DESTINATION lib
  LIBRARY DESTINATION lib
  RUNTIME DESTINATION bin)

---
#### Task 1.4: Add the Plugin to robot XACRO

1. In the **urdf** folder update the **conveyor_belt.urdf.xacro** file, add the following code before the last **</robot>** tag.


In [ ]:
<gazebo>
  <plugin filename="libconveyor_belt_plugin.so" name="conveyor_belt_plugin">
        <max_velocity>0.3</max_velocity> 
  </plugin>
</gazebo>

---
#### Task 1.5: Build and verify

1. Build the package, source the workspace and launch the Gazebo environment

In [ ]:
colcon build --packages-select conveyor_belt
source install/setup.bash
ros2 launch conveyor_belt robot.launch.py

**Expected Output**
![](images/init_and_load.png)


You can also verify the plugin being generated in your workspace in the **ros2_workspace/install/conveyor_belt/lib** folder. This is the name of plugin that we use while using it in the XACRO.

**Refer the below expected output**
![](images/lib_con.png)

***********
### Task 2: Add Connection Update for every time step in the simulation

To observe what happens in a plugin lets log some data and see how the connection behaves.

**Note: From here onwards we will be only updating the conveyor_belt_plugin.cpp file**



---
####  Task 2.1: Add event pointer to class

Connection to world update event. Callback is called while this is alive. Add the below pointer variable as a public member to the **ConveyorBeltPluginPrivate** class.


In [ ]:
gazebo::event::ConnectionPtr update_connection_;

---
#### Task 2.2: Create a Callback 

1. This callback function will  perform some task on update from the Gazebo world. Declare the method as a public method of the **ConveyorBeltPluginPrivate** class.

In [ ]:
void OnUpdate();

2. Add the method implementation to the code.

In [ ]:
void ConveyorBeltPluginPrivate::OnUpdate()
{

  std::cout<<CYAN<<"Gazebo World Updated"<<RESET<<std::endl;

}

---
#### Buiild, Source and Launch

In [ ]:
colcon build --packages-select conveyor_belt
source install/setup.bash
ros2 launch conveyor_belt robot.launch.py


**Expected Output**

![](images/update_connection.png)

**Note: Remove the Gazebo Update logging statement after verification**

-----------------
### Task 3: Get the Joint from model

We have to access the belt joint in order to move the link.

**Elements in Xacro**

Check the **urdf/conveyor_belt.urdf.xacro** file to see the joint tag and it's name as below.

```xml
<joint
    name="belt_joint"
    type="prismatic">
    <origin
      xyz="-2.83 0 0.3"
      rpy="1.5708 0 0" />
    <parent
      link="base_link" />
    <child
      link="belt_link" />
    <axis
      xyz="-1 0 0" />
    <limit
      lower="0"
      upper="0.01"
      effort="0"
      velocity="10" />
  </joint>
```

---
#### Task 3.1: Add Joint Pointer

The joint pointer is a variable that points to a specific joint from the SDF.

1. Declare the public member variable in the **ConveyorBeltPluginPrivate** class.

In [ ]:
gazebo::physics::JointPtr belt_joint_;

---
#### Task 3.2: Fetch and Verify the Joint

1. Get the joint from the model using joint name as mentioned in XACRO.
2. Add an exception handler to stop the plugin incase the joint is not found.

Add the blow code to the **ConveyorBeltPlugin::Load** method.

In [ ]:
impl_->belt_joint_ = model->GetJoint("belt_joint");

if (!impl_->belt_joint_) {
    std::cout<<RED<< "Belt joint not found, unable to start conveyor plugin"<<RESET<<std::endl;
    return;
}
else {
    std::cout <<GREEN<<"Belt joint found." << RESET <<std::endl;
}

---
#### Buiild, Source and Launch

In [ ]:
colcon build --packages-select conveyor_belt
source install/setup.bash
ros2 launch conveyor_belt robot.launch.py


**Expected Output**

![](images/belt_joint.png)

------------------------
### Task 4: Access values from URDF/SDF file of the model

In some cases we require information from the plugin passed as parameters such as wheel diameters, wheel base length, contant values for calculations, etc.


#### Xacro Snippet

1. We have some data embedded into our XACRO files like the limits of joints

```xml
<!-- Belt Joint -->
<joint
    name="belt_joint"
    type="prismatic">
    <origin
      xyz="-2.83 0 0.3"
      rpy="1.5708 0 0" />
    <parent
      link="base_link" />
    <child
      link="belt_link" />
    <axis
      xyz="-1 0 0" />
    <limit
      lower="0"
      upper="0.01"
      effort="0"
      velocity="10" />
  </joint>
```

2. Also we can pass parameters to our plugin from the xacro.

```xml
<!-- Plugin Attribute -->

<gazebo>
  <plugin filename="libconveyor_belt_plugin.so" name="conveyor_belt_plugin">
        <max_velocity>0.3</max_velocity> 
  </plugin>
</gazebo>

```

---
#### Task 4.1: Add memeber variables to store data

1. Declare public methods to the **ConveyorBeltPluginPrivate** class

In [ ]:
// Initial velocity of belt
double belt_velocity_= 0.5;
    
// Variable to store the max velocity from SDF
double max_velocity_;

/// Position limit of belt joint to reset (Get it from SDF)
double limit_;

---
#### Task 4.2: Get the details from the joint.

We use the **UpperLimit** method to get the joint limit. Add the below code to the **ConveyorBeltPluginPrivate::Load** method

In [ ]:
impl_->limit_ = impl_->belt_joint_->UpperLimit();
std::cout << PURPLE << "Max Joint Limit: "<<impl_->limit_<<RESET<<std::endl;

---
#### Task 4.3: Get the parameters from the plugin

We use the **GetElement** method to get the **max_velocity** parameter.  Add the below code to the **ConveyorBeltPluginPrivate::Load** method

In [ ]:
impl_->max_velocity_ = sdf->GetElement("max_velocity")->Get<double>();
std::cout<< PURPLE << "Max Velocity: "<<impl_->max_velocity_ <<RESET<<std::endl;

---
#### Buiild, Source and Launch

In [ ]:
colcon build --packages-select conveyor_belt
source install/setup.bash
ros2 launch conveyor_belt robot.launch.py


**Expected Output**

![](images/max_vel_limit.png)

---
### Task 5: Move the joints

We can now add the actual conveyor bel functionality to our plugin. We utlize all the information we get from the model and parameter.
1. Set the belt velocity to some value.
2. Check the belt joint position at each time step.
3. Set the position to 0 if the joint reached its limit.

Add the below code to the **ConveyorBeltPluginPrivate::OnUpdate** method.

In [ ]:
belt_joint_->SetVelocity(0, belt_velocity_);

double belt_position = belt_joint_->Position(0);

if (belt_position >= limit_){
    belt_joint_->SetPosition(0, 0);
}

##### Buiild, Source and Launch

In [ ]:
colcon build --packages-select conveyor_belt
source install/setup.bash
ros2 launch conveyor_belt robot.launch.py


**Expected Output**![](images/conveyor.gif)

--------------------
### Task 6: Adding ROS functionality

We often use Gazebo plugins in tandem with ROS so it is essential for us to understand how to add ROS functionalities into these plugins


---
#### Task 6.1: Member Declaration

1. Declare ROS node.
2. Declare a Subscriber
3. Declare the callback method

Add the following code to the **ConveyorBeltPluginPrivate** class.

In [ ]:
gazebo_ros::Node::SharedPtr ros_node_;

rclcpp::Subscription<std_msgs::msg::Bool>::SharedPtr power_subscriber;

void OnPower(const std_msgs::msg::Bool::SharedPtr msg);

---
#### Task 6.2: Initialize the node and subscriber

Add the following code to the **ConveyorBeltPlugin::Load** method

In [ ]:
// Create ROS node
impl_->ros_node_ = gazebo_ros::Node::Get(sdf);

// Add callback to subscriber
impl_->power_subscriber = impl_->ros_node_->create_subscription<std_msgs::msg::Bool>("conveyor_power", 
10, std::bind(&ConveyorBeltPluginPrivate::OnPower, impl_.get(), std::placeholders::_1));

---
#### Task 6.3: Add the callback declaration

Add the following code to the **ConveyorBeltPluginPrivate** class

In [ ]:
void ConveyorBeltPluginPrivate::OnPower(const std_msgs::msg::Bool::SharedPtr msg)
{
  if (msg->data == false){
    belt_velocity_ = 0;
    std::cout<<RED << "Conveyor Belt Powered Off"<<RESET<<std::endl;
  }
  else{
    belt_velocity_ = max_velocity_;
    std::cout<<GREEN << "Conveyor Belt Powered On"<<RESET<<std::endl;

  }
}

---
#### Buiild, Source and Launch

In [ ]:
colcon build --packages-select conveyor_belt
source install/setup.bash
ros2 launch conveyor_belt robot.launch.py

---
### Testing


0. ROS Topic Check

In [ ]:
ros2 topic list

**Expected Output**

![](images/topic_list.png)


1. Turn Off the Power

In [ ]:
ros2 topic pub /conveyor_power std_msgs/msg/Bool  '{data: false}' -1

2. Turn On the Power


In [ ]:
ros2 topic pub /conveyor_power std_msgs/msg/Bool  '{data: true}' -1

**Expected Ouptut**

![](images/on_belt.png)

---
### Future Work

1. Create the same thing for Gazebo Sim
2. Create other plugins such as Sensor or World

![](images/drone3.gif)